# pyprobasic

In [1]:
#hide
from nbdev.showdoc import *

## logging

## Logging with loguru

In [1]:
from loguru._logger import Core

In [2]:
c =Core()
lc = dir(c)
lc

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'activation_list',
 'activation_none',
 'enabled',
 'extra',
 'handlers',
 'handlers_count',
 'levels',
 'levels_ansi_codes',
 'lock',
 'min_level',
 'patcher']

In [3]:
c.levels.keys()

dict_keys(['TRACE', 'DEBUG', 'INFO', 'SUCCESS', 'WARNING', 'ERROR', 'CRITICAL'])

In [12]:

for a in lc:
    print('{}  {}'.format(a, eval('c.'+a)))

__class__  <class 'loguru._logger.Core'>
__delattr__  <method-wrapper '__delattr__' of Core object at 0x7fe57c24cc10>
__dict__  {'levels': {'TRACE': Level(name='TRACE', no=5, color='<cyan><bold>', icon='✏️'), 'DEBUG': Level(name='DEBUG', no=10, color='<blue><bold>', icon='🐞'), 'INFO': Level(name='INFO', no=20, color='<bold>', icon='ℹ️'), 'SUCCESS': Level(name='SUCCESS', no=25, color='<green><bold>', icon='✔️'), 'WARNING': Level(name='WARNING', no=30, color='<yellow><bold>', icon='⚠️'), 'ERROR': Level(name='ERROR', no=40, color='<red><bold>', icon='❌'), 'CRITICAL': Level(name='CRITICAL', no=50, color='<RED><bold>', icon='☠️')}, 'levels_ansi_codes': {'TRACE': '\x1b\x1b', 'DEBUG': '\x1b\x1b', 'INFO': '\x1b', 'SUCCESS': '\x1b\x1b', 'WARNING': '\x1b\x1b', 'ERROR': '\x1b\x1b', 'CRITICAL': '\x1b\x1b', None: ''}, 'handlers_count': count(0), 'handlers': {}, 'extra': {}, 'patcher': None, 'min_level': inf, 'enabled': {}, 'activation_list': [], 'activation_none': True, 'lock': <unlocked _thread.lo

In [4]:
from loguru import logger

logger.info("Straightforward logging. Replace your informative print statements")
logger.info("Continue logging.")
logger.info("and Yet Again Continue logging.")

2021-06-20 22:02:05.023 | INFO     | __main__:<module>:3 - Straightforward logging. Replace your informative print statements
2021-06-20 22:02:05.025 | INFO     | __main__:<module>:4 - Continue logging.
2021-06-20 22:02:05.026 | INFO     | __main__:<module>:5 - and Yet Again Continue logging.


In [5]:
def betterThanPrint(msg):
    logger.error("cool: {}".format(msg))
betterThanPrint('logging')

2021-06-20 22:02:09.617 | ERROR    | __main__:betterThanPrint:2 - cool: logging


In [6]:
x =1
logger.debug("x:{}".format(x))

2021-06-20 22:02:20.733 | DEBUG    | __main__:<module>:2 - x:1


logger.debug(_message, *args, **kwargs)
logger.info(_message, *args, **kwargs)
logger.success(_message, *args, **kwargs)
logger.warning(_message, *args, **kwargs)
logger.error(_message, *args, **kwargs)
logger.critical(_message, *args, **kwargs)

## LoggingState

In [2]:
#export
from dataclasses import dataclass
from loguru import logger

@dataclass
class LoggingState:
    states =  ['debug', 'info', 'success', 'warning', 'critical']
    debug: bool  = False
    info: bool = False
    success: bool  = False
    warning: bool  = False
    error: bool = True
    critical: bool  = False

In [8]:
LogInfo = LoggingState(info=True)
LogInfoDebug = LoggingState(info=True,debug=True)
LogDebugOn = LoggingState(debug=True)
LogDebugOff = LoggingState(debug=False)
LogWarning = LoggingState(warning=True)
LogCritical = LoggingState(critical=True)

In [9]:
f = LogInfo
X = str(f)

eval(X+'.info')

True

In [10]:
for key in  f.states :
    print(key)

debug
info
success
warning
critical


## log_call Decorator

In [11]:
from typing import Dict, List, Any

def log_output(logState:LoggingState, fun, result:Any) -> None:
    for key in  logState.states :
        if  eval(str(logState)+'.'+key):
            eval('logger.'+key)("function: {}, result:{}".format(fun.__name__,result))


In [12]:
def log_call(logState):
    """
        Decorator @log_call wraps the function 
        with log events.
    """
    def decorator(fun):
        @wraps(fun)
        def wrapper(*args, **kwargs):
            #Pre
            result = fun(*args, **kwargs)
            #post:
            log_output(logState,fun, result)
            return result
        return wrapper
    return decorator

In [88]:
LogDebug = LogDebugOn
@log_call(LogDebug)
def add_one(x):
    x = x+1
    return(x)

y=0
y =add_one(y)
y = add_one(y)
add_one(y)

2021-06-02 17:43:49.515 | DEBUG    | __main__:log_output:4 - function: add_one, result:1
2021-06-02 17:43:49.518 | DEBUG    | __main__:log_output:4 - function: add_one, result:2
2021-06-02 17:43:49.520 | DEBUG    | __main__:log_output:4 - function: add_one, result:3


3

## log_error Decorator

In [ ]:
def log_error(logState):
    """
        Decorator @log_call wraps the function 
        with log events.
    """
    def decorator(fun):
        @wraps(fun)
        def wrapper(*args, **kwargs):
            #Pre
            if fun(*args, **kwargs):
                continue
            else:
            #post:
            log_output(logState,fun, result)
            return result
        return wrapper
    return decorator

## Increment, decrement

In [92]:
@dataclass
class Counter:
    count: int = 0

In [96]:
count99 = Counter(count=99)
count99.count

99

In [91]:
call_count= 0
def increment(fun, increase):
    """
            Decorator pre-function call and post-function call of function func.
    """
    @wraps(fun)
    def wrapper(*args, **kwargs):
        global call_count
        #Pre action
        result = fun(*args, **kwargs)
        logger.info('call_count:{}'.format(call_count))
        if increase: call_count =+ 1
        else: call_count =- 1
        return result
    return wrapper

In [ ]:
@log_call
@add_one
def pow(x,y):
    return(x**y)

In [ ]:
print('',call_count)
print(pow(2,10))

In [ ]:
print('',call_count)
print(pow(2,10))

In [ ]:
from typing import Dict, List, Any
def log_output(fun, result:Any, kw:Dict, check_state:List ) -> None:
    for key in kw:
        keyl = key.lower()
        if  keyl in check_state:
            if kw[key]:
                eval('logger.'+keyl)("function: {}, result:{}".format(fun.__name__,result))

def log_call(*a, **kw):

    """
        Decorator @log_call causes the wrapped funtion 
        to log at state given in **kw the 
        "fucntion: {}, result:{}".format(fun.__name__,result)

    """

    def decorator(fun):
        @wraps(fun)
        def wrapper(*args, **kwargs):
            #Pre
            result = fun(*args, **kwargs)
            #post:
            check_state = ('debug', 'info', 'success', 'warning', 'error', 'critical')
            log_output(fun, result, kw, check_state)
            return result
        return wrapper
    return decorator

In [ ]:
@log_call(ERROR=True)
@add_one
def pow(x,y):
    return(x**y)

In [ ]:
print(pow(2,10))
print('',call_count)

In [ ]:
@add_one
@log_call(ERROR=True)
def pow(x,y):
    return(x**y)

In [ ]:
print('',call_count)
print(pow(2,100))
